In [2]:
# encoding=utf-8
import jieba

jieba.enable_paddle()# 启动paddle模式。 0.40版之后开始支持，早期版本不支持
strs=["我来到北京清华大学","乒乓球拍卖完了","中国科学技术大学"]
for str in strs:
    seg_list = jieba.cut(str,use_paddle=True) # 使用paddle模式
    print("Paddle Mode: " + '/'.join(list(seg_list)))


In [3]:
strs=["我来到北京清华大学","乒乓球拍卖完了","中国科学技术大学"]
for str in strs:
    seg_list = jieba.cut(str,use_paddle=False) # 使用paddle模式
    print("Paddle Mode: " + '/'.join(list(seg_list)))

In [ ]:


seg_list = jieba.cut("我来到北京清华大学", cut_all=True)
print("Full Mode: " + "/ ".join(seg_list))  # 全模式

seg_list = jieba.cut("我来到北京清华大学", cut_all=False)
print("Default Mode: " + "/ ".join(seg_list))  # 精确模式

seg_list = jieba.cut("他来到了网易杭研大厦")  # 默认是精确模式
print(", ".join(seg_list))

seg_list = jieba.cut_for_search("小明硕士毕业于中国科学院计算所，后在日本京都大学深造")  # 搜索引擎模式
print(", ".join(seg_list))

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
import requests
from models.clip_model import CLIPModel

# 数据集路径
data_dir = 'data/flickr30k'

# 设置数据集
from data_setup import setup_flickr30k_dataset
setup_flickr30k_dataset()

# 初始化自定义CLIP模型
model = CLIPModel()
device = model.device  # 使用模型中已定义的设备
preprocess = model.image_transform  # 使用模型中已定义的预处理


In [ ]:

# Flick30K Dataset  
class Flickr30KDataset(Dataset):  
    def __init__(self, root_dir, annotations_file, transform=None):  
        self.root_dir = root_dir  
        self.annotations = pd.read_csv(annotations_file, sep='\t')  
        self.annotations.columns = ["image", "caption"]  
        self.transform = transform  

    def __len__(self):  
        return len(self.annotations)  

    def __getitem__(self, idx):  
        img_path = os.path.join(self.root_dir, self.annotations.iloc[idx, 0])  
        image = Image.open(img_path).convert("RGB")  

        if self.transform:  
            image = self.transform(image)  

        caption = self.annotations.iloc[idx, 1]  

        return image, caption  

# 定义数据集和数据加载器  
transform = transforms.Compose([  
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  
])  

dataset = Flickr30KDataset(root_dir=os.path.join(data_dir, 'flickr30k_images'),   
                           annotations_file=os.path.join(data_dir, 'results_20130124.token'),   
                           transform=transform)  

data_loader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)  